## Import libraries 

In [9]:
from multiprocessing import Pool
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

## Define functions 

In [10]:
def arrangement_1(nleds,d):
    l=math.sqrt(nleds);
    ledcord=np.zeros((2,nleds));
    temp=0;
    for k in range(nleds):
        ledcord[0,k]=-(l-1)*d/2 +((k)%l)*d;
        if (k)%l ==0 and k != 0:
            temp=temp+1;
        ledcord[1,k]=-(l-1)*d/2 + temp*d;
    return ledcord

def arrangement_2(nleds,d):
    l=math.sqrt(nleds);
    ledcord=np.zeros((2,nleds));
    temp=0;
    for k in range(nleds):
        if (k)%2 ==0:
            ledcord[0,k]=-(l-1)*d/2 +((k)%l)*d;
        elif (k)%2 ==1:
            ledcord[0,k]=-(l-2)*d/2 +((k)%l)*d;
        
        if (k)%l ==0 and k != 0:
            temp=temp+1;
        
        ledcord[1,k]=-(l-1)*d/2 + temp*d;
    return ledcord

def E_r(ledcord,nleds,Ir_0,z,d,r):    
    x=r;
    y=0;
    E=0;
    for i in range(nleds):
        den=np.square(np.square(x-ledcord[0,i]) +np.square(y-ledcord[1,i]) +z**2);
        E= E + (z**2 * Ir_0)*(np.reciprocal(den))
    return E
  

# Arrangement 1 

## Percent_Uniformity vs d and z 

In [11]:
Ir_0 = 0.1
final_list = []

def ratio(d):
    final_list = []
    Ir_0=0.1
    for nleds in [9,25,49,81,121]:
        lled=np.sqrt(nleds)   
        ledcord = arrangement_1(nleds,d)
        nz = round((0.5 - d)/0.001) + 1
        for z in np.linspace(d,0.5,nz):
            min_E=1000000;
            max_E=0;
            ni = round((0.04)/0.001) + 1
            for i in np.linspace(0,0.04,ni):
                E_i = E_r(ledcord,nleds,Ir_0,z,d,i)
                min_E = min(min_E,E_i)
                max_E = max(max_E,E_i)
            percent_uniformity=(min_E/max_E)*100

            t = 4000/(max_E + min_E)
            energy_max = max_E * t
            energy_min = min_E * t

            final_list.append([nleds,d,z,min_E,max_E,t,energy_min/10,energy_max/10,percent_uniformity])
    return final_list

with Pool(7) as p:
    l = list(tqdm(p.imap(ratio,np.linspace(0.01,0.2,191)), total=191))

100%|██████████| 191/191 [45:43<00:00, 14.36s/it] 


In [12]:
t=[]
for i in l:
    t.extend(i)
#     print(len(i))
final_list=t

In [13]:
df_1 = pd.DataFrame(final_list,columns=["nleds","d(m)","z(m)","min_E(W/m^2)","max_E(W/m^2)","exposure_time(s)","energy_min(mJ/cm^2)","energy_max(mJ^cm^2)","percent_uniformity"])
#pd.set_option("display.max_rows", None, "display.max_columns", None)
#df.pivot_table(index = ['d(m)','nleds'], aggfunc ='size') 

## Maximum_percent_uniformity 

In [14]:
idx = df_1.groupby(['nleds'])['percent_uniformity'].transform(max) == df_1['percent_uniformity']
df_1[idx].sort_values("nleds")

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity
376975,9,0.2,0.500,2.492124,2.507334,800.086610,199.391541,200.608459,99.393387
377116,25,0.2,0.340,5.751994,5.764446,347.329552,199.783750,200.216250,99.783984
377437,49,0.2,0.360,6.480728,6.486233,308.476272,199.915095,200.084905,99.915131
377758,81,0.2,0.380,6.866439,6.869194,291.213385,199.959881,200.040119,99.959889
378078,121,0.2,0.399,7.097583,7.099109,281.755781,199.978503,200.021497,99.978505


## Minimum_exposure_time

In [17]:
ef_1=df_1[df_1['percent_uniformity']>99]
idx = ef_1.groupby(['nleds'])['exposure_time(s)'].transform(min) == ef_1['exposure_time(s)']
ef_1[idx].sort_values("nleds")

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity
339128,9,0.176,0.249,6.034938,6.095878,329.738733,198.995277,201.004723,99.000299
185160,25,0.092,0.138,28.695536,28.985091,69.347373,198.996007,201.003993,99.001022
90694,49,0.048,0.061,122.840000,124.072012,16.200103,199.002065,200.997935,99.007019
56612,81,0.033,0.041,270.900817,273.583297,7.346404,199.014671,200.985329,99.019502
38214,121,0.025,0.035,474.073674,478.809737,4.197785,199.005951,200.994049,99.010867


# Arrangement 2

In [18]:
Ir_0 = 0.1
final_list = []

def ratio(d):
    final_list = []
    Ir_0=0.1
    for nleds in [9,25,49,81,121]:
        lled=np.sqrt(nleds)   
        ledcord = arrangement_2(nleds,d)
        nz = round((0.5 - d)/0.001) + 1
        for z in np.linspace(d,0.5,nz):
            min_E=1000000;
            max_E=0;
            ni = round((0.04)/0.001) + 1
            for i in np.linspace(0,0.04,ni):
                E_i = E_r(ledcord,nleds,Ir_0,z,d,i)
                min_E = min(min_E,E_i)
                max_E = max(max_E,E_i)
            percent_uniformity=(min_E/max_E)*100

            t = 4000/(max_E + min_E)
            energy_max = max_E * t
            energy_min = min_E * t

            final_list.append([nleds,d,z,min_E,max_E,t,energy_min/10,energy_max/10,percent_uniformity])
    return final_list

with Pool(7) as p:
    l = list(tqdm(p.imap(ratio,np.linspace(0.01,0.2,191)), total=191))

100%|██████████| 191/191 [48:17<00:00, 15.17s/it] 


In [19]:
t=[]
for i in l:
    t.extend(i)
#     print(len(i))
final_list=t

In [20]:
df_2 = pd.DataFrame(final_list,columns=["nleds","d(m)","z(m)","min_E(W/m^2)","max_E(W/m^2)","exposure_time(s)","energy_min(mJ/cm^2)","energy_max(mJ^cm^2)","percent_uniformity"])
#pd.set_option("display.max_rows", None, "display.max_columns", None)
#df.pivot_table(index = ['d(m)','nleds'], aggfunc ='size') 

## Maximum_percent_uniformity

In [22]:
idx = df_2.groupby(['nleds'])['percent_uniformity'].transform(max) == df_2['percent_uniformity']
df_2[idx].sort_values("nleds")

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity
376781,9,0.2,0.306,4.367443,4.376950,457.436000,199.782551,200.217449,99.782787
377127,25,0.2,0.351,5.649510,5.653269,353.895270,199.933493,200.066507,99.933515
377466,49,0.2,0.389,6.294962,6.296694,317.670684,199.972476,200.027524,99.972480
377799,81,0.2,0.421,6.675072,6.675946,299.602627,199.986911,200.013089,99.986912
378127,121,0.2,0.448,6.923597,6.924103,288.856633,199.992698,200.007302,99.992698


## Minimum_exposure_time

In [2]:
ef_2=df_2[df_2['percent_uniformity']>99]
idx = ef_2.groupby(['nleds'])['exposure_time(s)'].transform(min) == ef_2['exposure_time(s)']
ef_2[idx].sort_values("nleds")

NameError: name 'df_2' is not defined

In [1]:
ef_2

NameError: name 'ef_2' is not defined